In [1]:
from pmd_beamphysics import ParticleGroup
from pmd_beamphysics.interfaces.elegant import elegant_to_data
from h5py import File


import numpy as np

from pytao import Tao

import subprocess
import os

root_dir = os.getcwd()

## Helper functions

In [2]:
# Make sure root_dir/code has been added to $PTHONPATH
from parse_output import get_twiss_tao, get_twiss_elegant

# Tao

#### prepare bmad lattice file

In [64]:
quad_dict_bmad={'L':0.03, 'k1':2.0, 'offset':0.0}

In [65]:
def write_bmad_lattice_file(ele_type, dict_bmad):
    
    out1 = f"""parameter[particle] = electron
no_digested

beginning[beta_a] = 10
beginning[alpha_a] = 1
beginning[beta_b] = 10
beginning[alpha_b] = 1

parameter[geometry] = open
parameter[p0c] = 10e6

ele1: {ele_type}"""
    
    out2 = ""
    for key, value in dict_bmad.items():
        out2 += f",\n{key} = {value}"
    
    out3 = """\n
lat: line = (ele1)

use, lat
    """
    
    f = open(root_dir+"/bmad/one_ele.bmad", "wt")
    f.write(out1)
    f.write(out2)
    f.write(out3)
    f.close()
    
    return None
    

In [66]:
write_bmad_lattice_file('quad', quad_dict_bmad)

#### Prepare elegant lattice file

In [61]:
quad_dict_elegant={'L':0.03, 'K1':2.0}

In [62]:
def write_elegant_lattice(ele_type, dict_elegant):
    
    out1=f"""C: CHARGE,TOTAL= 13E-12
    
ELE1: {ele_type}"""

    out2 =""
    for key, value in dict_elegant.items():
        out2 += f",\n{key} = {value}"

    out3 = """\n
BPM_END: WATCH,FILENAME="BPM_END.out"

MYLINE: LINE = (C, ELE1, BPM_END)
    """
    
    f = open(root_dir+"/elegant/one_ele.lte", "wt")
    f.write(out1)
    f.write(out2)
    f.write(out3)
    f.close()        

    return None

In [63]:
write_elegant_lattice("QUAD", quad_dict_elegant)

# run Tao (bmad)

In [6]:
# This line can only be run ONCE
tao = Tao(f'-init {root_dir}/bmad/tao.init -noplot')

In [7]:
tao.cmd('beamon')
tao.cmd('write beam ./bmad/beam_end_tao.hdf5 -at END');  # For analysis

#### collect Bmad beam and Twiss

In [8]:
H5FILE = root_dir+'/bmad/beam_end_tao.hdf5'
Ptao = ParticleGroup(H5FILE)

twiss_tao = get_twiss_tao(tao, 'END')

# run Elegant

In [50]:
# Make sure elegant has been installed and located
!which elegant

/Users/wlou/Code/elegant/bin/elegant


In [51]:
os.chdir(root_dir+'/elegant')
subprocess.run(["elegant", "one_ele.ele"], stdout=subprocess.DEVNULL)
#os.chdir(root_dir)

CompletedProcess(args=['elegant', 'one_ele.ele'], returncode=0)

#### collect Elegant beam and Twiss

In [52]:
OUTFILE = root_dir+'/elegant/BPM_END.out'
DAT = elegant_to_data(OUTFILE, charge = 13e-12)
Pele = ParticleGroup(data=DAT)

elegant_twiss_file_name = root_dir + '/elegant/one_ele.twi'
twiss_elegant = get_twiss_elegant(elegant_twiss_file_name)

# Compare beams and Twiss from Bmad and elegant

In [53]:
twiss_elegant - twiss_tao

array([ 0.07773734, -0.58968962,  0.04188112,  0.6031253 ,  0.        ,
        0.        ,  0.        ,  0.        ])

In [46]:
twiss_tao

array([9.92226266, 1.58968962, 9.95811888, 0.3968747 , 0.        ,
       0.        , 0.        , 0.        ])

In [47]:
twiss_elegant

array([10.,  1., 10.,  1.,  0.,  0.,  0.,  0.])

In [20]:
def compare_twiss(twiss_tao, twiss_elegant, tol=1e-6):
    print("Agree in all 8 Twiss...?", np.all(abs(twiss_tao - twiss_elegant) < tol))
    return None

In [18]:
compare_twiss(twiss_tao, twiss_elegant, 1e-6)

Agree in all 8 Twiss...? True


In [54]:
def compare_two_P(Ptao, Pele, tol=1e-9):
    
    assert Ptao.n_particle == Pele.n_particle, "The two particle groups need to have the same number of particles"
    
    print(f"Comparing {Ptao.n_particle} particles...")
    
    print("Agree in all x...?", np.all(abs(Ptao.x - Pele.x) < tol))
    print("Agree in all xp...?", np.all(abs(Ptao.xp - Pele.xp) < tol))
    print("Agree in all y...?", np.all(abs(Ptao.y - Pele.y) < tol))
    print("Agree in all yp...?", np.all(abs(Ptao.yp - Pele.yp) < tol))
    print("Agree in all t...?", np.all(abs(Ptao.t - Pele.t) < tol))
    
    print("Agree in all delta (E0=10MeV)?", np.all(abs(Ptao.energy - Pele.energy)/10E6 < tol))
    

In [55]:
compare_two_P(Ptao, Pele, tol=1e-9)

Comparing 13 particles...
Agree in all x...? False
Agree in all xp...? False
Agree in all y...? False
Agree in all yp...? False
Agree in all t...? True
Agree in all delta (E0=10MeV)? True


In [56]:
Pele.x

array([ 0.   , -0.001,  0.001,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
        0.   ,  0.   ,  0.   ,  0.   ,  0.   ])

In [13]:
%%tao
write beam beam0_13p.hdf5 -at BEGINNING

-------------------------
Tao> write beam beam0_13p.hdf5 -at BEGINNING
[INFO] tao_write_cmd:
    Written: beam0_13p.hdf5
-------------------------
Tao> 


In [14]:
!pwd

cmake(8):ERROR:105: Unable to locate a modulefile for 'cmake/3.18.2'
/global/u2/w/wlou1991/bmad_vs_elegant_simple
